# pySpark Commands Reference
http://spark.apache.org/docs/2.1.3/api/python/index.html


## Connect to Spark Cluster
https://github.com/minrk/findspark

In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("pyspark-df-overview").getOrCreate()

In [ ]:
spark.version

### Create Spark DataFrame
Dataset from: https://www.kaggle.com/uciml/adult-census-income/home

In [ ]:
df = spark.read.csv("/dataset/pyspark-df-overview/census_income.csv.gz", header=True)
df.printSchema()

### Define a schema

In [ ]:
import pyspark.sql.types as t

census_schema = t.StructType([
      t.StructField('age', t.IntegerType(), True)
    , t.StructField('workclass', t.StringType(), True)
    , t.StructField('fnlwgt', t.IntegerType(), True)
    , t.StructField('education', t.StringType(), True)
    , t.StructField('education-num', t.IntegerType(), True)
    , t.StructField('marital-status', t.StringType(), True)
    , t.StructField('occupation', t.StringType(), True)
    , t.StructField('relationship', t.StringType(), True)
    , t.StructField('race', t.StringType(), True)
    , t.StructField('sex', t.StringType(), True)
    , t.StructField('capital-gain', t.DoubleType(), True)
    , t.StructField('capital-loss', t.DoubleType(), True)
    , t.StructField('hours-per-week', t.IntegerType(), True)
    , t.StructField('native-country', t.StringType(), True)
    , t.StructField('label', t.StringType(), True)
])

In [ ]:
# Support for compressed (gziped) payload
df = spark.read.csv("/dataset/pyspark-df-overview/census_income.csv.gz", header=True, schema=census_schema)
df.printSchema()

In [ ]:
df.count()

### Drop unused column

In [ ]:
df = df.drop('fnlwgt')
df.printSchema()

### Few operations

In [ ]:
from pyspark.sql.functions import count, avg, desc

df.groupBy(['education']). \
agg(
    count('*').alias('qty'), 
    avg('age').alias('avg_age')
).orderBy(desc('qty')). \
show()

In [ ]:
df.select('age', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week').describe().show()

In [ ]:
df.select('workclass', 'education', 'marital-status').describe().show()

In [ ]:
df.freqItems(['marital-status']).show(truncate=False)

### Check if there is missing data

In [ ]:
from pyspark.sql.functions import isnan, when, count, col

# All columns
# cols = df.columns
# Selected columns
cols = ['workclass', 'education-num', 'occupation', 'hours-per-week', 'native-country']

# https://stackoverflow.com/a/44631639/570393
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in cols]).show()

### Remove rows with missing data

In [ ]:
# Total rows
print('total rows: %s' % df.count())

# After droping NA records
print('only complete rows: %s' % df.dropna().count())

### Fill rows that contains missing data

In [ ]:
def show_df(df, field='occupation'):
    df.groupBy(field).count().show()

In [ ]:
show_df(df)

In [ ]:
# Fill with a fixed value
new_df = df.fillna({'occupation': 'Other-service'})

# Count 
show_df(new_df)

### Better way

Calc the `mean()` value of a column and use it on missing values.
Also use a static string for categorical data 

In [ ]:
from pyspark.sql.functions import mean
import pandas as pd

data_to_fill = \
    df.groupBy().agg(mean('hours-per-week').alias('hours-per-week')).toPandas().to_dict('records')[0]

# Simple Python Dict Update
data_to_fill.update({'occupation': 'Other-service'})


df.fillna(data_to_fill).show()

### Creating charts with pandas & matplotlib
https://pandas.pydata.org/pandas-docs/stable/api.html#api-dataframe-plotting

**Important:** possible only when data become small enough to driver program

In [ ]:
# This is distributed
df_spark = df.groupBy('workclass').agg(count('*').alias('counts')).orderBy('counts')

# This is running on driver
df_wk = df_spark.toPandas()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

df_wk.plot.bar(x='workclass', y='counts', figsize=(20,6));

### Stop Drive Program
Release resources from Spark Cluster

In [ ]:
spark.stop()

# Continue Learning

* [Kaggle Learn](https://www.kaggle.com/learn/overview)
* [PySpark Cookbook](https://www.safaribooksonline.com/library/view/pyspark-cookbook/9781788835367/)